# IIC2115 - Programación Como Herramienta para la Ingeniería
## Taller 3b
### Ayudante: Matías Gaete Silva - mzgaete@uc.cl
Importamos librerías necesarias:

In [14]:
import sqlite3

#### Consulta: Para todos los niveles, con excepción de JR, encuentre el nivel y el promedio de edad de los estudiantes del nivel.
De la tabla Estudiantes agrupamos por nivel (excepto para el nivel JR) y seleccionamos el nivel junto con el promedio de edad de los estudiantes del nivel.

In [15]:
connection = sqlite3.connect('data.db')
cursor = connection.cursor()
cursor.execute("SELECT E.nivel, AVG(E.edad) \
                FROM Estudiantes E \
                WHERE E.nivel != 'JR' \
                GROUP BY E.nivel")
print(cursor.fetchall())
connection.close()

[('FR', 17.666666666666668), ('SO', 18.4), ('SR', 20.714285714285715)]


#### Consulta: Encuentre los nombres de todos los alumnos que no están inscritos en ningún curso.
Los alumnos inscritos se encuentran en la tabla Inscritos, por lo que en la consulta anidada obtenemos los alumnos que están inscritos en algún curso. Con esta información, de la tabla Estudiantes seleccionamos el nombre de los estudiantes que no se encuentran en el resultado anterior, lo que nos otorgará los nombres de los alumnos que no están inscritos en ningún curso.

In [16]:
connection = sqlite3.connect('data.db')
cursor = connection.cursor()
cursor.execute("SELECT E.nombre \
                FROM Estudiantes E \
                WHERE E.num NOT IN (SELECT I.num_est \
                                    FROM Inscritos I)")
print(cursor.fetchall())
connection.close()

[('Maria White',), ('Charles Harris',), ('Angela Martinez',), ('Thomas Robinson',), ('Margaret Clark',), ('Dorthy Lewis',), ('Daniel Lee',), ('Nancy Allen',), ('Mark Young',), ('Donald King',), ('George Wright',), ('Steven Green',), ('Edward Baker',)]


#### Consulta: Encuentre los nombres de los profesores para los cuales la suma de estudiantes de todos los cursos que dictan es menor que 5 (considere solo aquellos cursos con al menos 1 estudiante inscrito).

Notemos que Inscritos tiene información de los cursos y los estudiantes inscritos (la necesitamos para obtener la cantidad de alumnos por curso); Cursos tiene información sobre los cursos y profesores que lo dictan (la necesitamos para saber qué profesores dictan qué cursos); y Profesores tiene información sobre los nombres de estos (que es lo que queremos retornar, ya que Cursos usa el id de los profesores). Por lo tanto, debemos hacer un JOIN entre estas tablas para tener toda la información que necesitamos. Con lo anterior, basta agrupar por profesor (usando GROUP BY) e imponer la condición de que la suma de estudiantes de todos los cursos que dictan es menor que 5 (con COUNT se obtiene la cantidad de alumnos y con HAVING se impone la condición).

In [17]:
connection = sqlite3.connect('data.db')
cursor = connection.cursor()
cursor.execute("SELECT P.nombre \
                FROM Profesores P, Cursos C, Inscritos I \
                WHERE P.id = C.id_profesor AND I.nombre_curso = C.nombre \
                GROUP BY P.id \
                HAVING COUNT(I.num_est)<5")
print(cursor.fetchall())
connection.close()

[('John Williams',), ('Elizabeth Taylor',), ('Mary Johnson',), ('William Moore',), ('Patricia Jones',), ('Richard Jackson',), ('Jennifer Thomas',)]


Otra forma:

In [18]:
connection = sqlite3.connect('data.db')
cursor = connection.cursor()
cursor.execute("SELECT P.nombre \
                FROM Profesores P \
                WHERE P.id IN (SELECT C.id_profesor \
                               FROM Cursos C, Inscritos I \
                               WHERE C.nombre = I.nombre_curso \
                               GROUP BY C.id_profesor \
                               HAVING COUNT(I.num_est)<5)")
print(cursor.fetchall())
connection.close()

[('John Williams',), ('Elizabeth Taylor',), ('Mary Johnson',), ('William Moore',), ('Patricia Jones',), ('Richard Jackson',), ('Jennifer Thomas',)]


#### Consulta: Encuentre los nombres de todos los cursos que tienen cátedra en la sala R128 o tienen cinco o más estudiantes.
Primero, los cursos que tienen cinco o más estudiantes los podemos obtener de Inscritos, agrupando por curso e imponiendo que la cantidad de estudiantes sea mayor o igual a 5 (usando COUNT para obtener la cantidad de estudiantes y con HAVING imponemos la condición). Esto es lo que representa la consulta anidada. Con esto, de la tabla Cursos seleccionamos el nombre de los cursos cuya sala es la R128 o que están en el resultado de la primera consulta discutida (cursos con cinco o más estudiantes).

In [19]:
connection = sqlite3.connect('data.db')
cursor = connection.cursor()
cursor.execute("SELECT C.nombre \
                FROM Cursos C \
                WHERE C.sala = 'R128' OR C.nombre IN (SELECT I.nombre_curso \
                                                      FROM Inscritos I \
                                                      GROUP BY I.nombre_curso \
                                                      HAVING COUNT(I.num_est) >= 5)")
print(cursor.fetchall())
connection.close()

[('Data Structures',), ('Database Systems',), ('Operating System Design',), ('Archaeology of the Incas',), ('Patent Law',), ('Dairy Herd Management',), ('Intoduction to Math',)]
